<a href="https://colab.research.google.com/github/chhak/DeepLearning/blob/master/Chatbot_create_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요 패키지 임포트
import codecs
import tensorflow as tf
import keras
import numpy as np
import random, sys
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from bs4 import BeautifulSoup

In [ ]:
# 파일 로드 함수정의
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines:
      data = line.split(',')
      data = data[0] + ' ' + data[1]
      result.append(data)

  result = result[1:] # header정보 제외
  return result

In [ ]:
# 데이터 로드
chat_dataset = load_data('/content/drive/My Drive/Tensorflow_works/data/ChatbotData.csv')
chat_text = ' '.join(chat_dataset)
chat_text

In [ ]:
# 문자 벡터화
chars = sorted(list(set(chat_text)))
char_index = dict((c, i) for i, c in enumerate(chars)) # 문자 - index
index_char = dict((i, c) for i, c in enumerate(chars)) # index - 문자

print(len(chars))
print(index_char)

1241
{0: ' ', 1: '!', 2: '"', 3: '%', 4: "'", 5: '-', 6: '.', 7: '0', 8: '1', 9: '2', 10: '3', 11: '4', 12: '5', 13: '6', 14: '7', 15: '8', 16: '9', 17: ';', 18: '?', 19: 'A', 20: 'B', 21: 'C', 22: 'D', 23: 'L', 24: 'N', 25: 'O', 26: 'P', 27: 'S', 28: 'X', 29: '_', 30: 'a', 31: 'c', 32: 'g', 33: 'j', 34: 'k', 35: 'n', 36: 'o', 37: 's', 38: '~', 39: '…', 40: 'ㅊ', 41: 'ㅋ', 42: 'ㅎ', 43: 'ㅜ', 44: 'ㅠ', 45: '가', 46: '각', 47: '간', 48: '갇', 49: '갈', 50: '감', 51: '갑', 52: '값', 53: '갔', 54: '강', 55: '갖', 56: '같', 57: '갚', 58: '개', 59: '객', 60: '갠', 61: '갯', 62: '갱', 63: '걍', 64: '걔', 65: '거', 66: '걱', 67: '건', 68: '걷', 69: '걸', 70: '검', 71: '겁', 72: '것', 73: '겉', 74: '게', 75: '겐', 76: '겜', 77: '겟', 78: '겠', 79: '겨', 80: '격', 81: '겪', 82: '견', 83: '결', 84: '겹', 85: '겼', 86: '경', 87: '곁', 88: '계', 89: '곗', 90: '고', 91: '곡', 92: '곤', 93: '곧', 94: '골', 95: '곰', 96: '곱', 97: '곳', 98: '공', 99: '과', 100: '관', 101: '광', 102: '괘', 103: '괜', 104: '괴', 105: '교', 106: '구', 107: '국', 108: '군', 109: '굳', 110:

In [ ]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록
maxlen = 20
step = 3
sentences = []
next_char = []

for i in range(0, len(chat_text) - maxlen, step):
  sentences.append(chat_text[i: i + maxlen])
  next_char.append(chat_text[i + maxlen])



In [ ]:
#next_char
train_x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
train_y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):

  for t, char in enumerate(sentence):
    train_x[i, t, char_index[char]] = 1

  train_y[i, char_index[next_char[i]]] = 1

train_x

In [ ]:
# 모델 구성
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

In [ ]:
# 모델 학습
model.fit(train_x, 
          train_y, 
          batch_size=128, 
          epochs=10)

model

In [ ]:
# 모델 저장
model.save('/content/drive/My Drive/Tensorflow_works/model/chatbot.model')